In [169]:
import altair as alt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

##### Load the Data

In [170]:
df = pd.read_csv(r'C:\Users\Blake Dennett\Downloads\Summer2023\loan_approval_dataset.csv')
df.head()

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


In [171]:
df.describe()

,loan_id,no_of_dependents,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value
count,4269.000000,4269.000000,4.269000e+03,4.269000e+03,4269.000000,4269.000000,4.269000e+03,4.269000e+03,4.269000e+03,4.269000e+03
mean,2135.000000,2.498712,5.059124e+06,1.513345e+07,10.900445,599.936051,7.472617e+06,4.973155e+06,1.512631e+07,4.976692e+06
std,1232.498479,1.695910,2.806840e+06,9.043363e+06,5.709187,172.430401,6.503637e+06,4.388966e+06,9.103754e+06,3.250185e+06
min,1.000000,0.000000,2.000000e+05,3.000000e+05,2.000000,300.000000,-1.000000e+05,0.000000e+00,3.000000e+05,0.000000e+00
25%,1068.000000,1.000000,2.700000e+06,7.700000e+06,6.000000,453.000000,2.200000e+06,1.300000e+06,7.500000e+06,2.300000e+06
50%,2135.000000,3.000000,5.100000e+06,1.450000e+07,10.000000,600.000000,5.600000e+06,3.700000e+06,1.460000e+07,4.600000e+06
75%,3202.000000,4.000000,7.500000e+06,2.150000e+07,16.000000,748.000000,1.130000e+07,7.600000e+06,2.170000e+07,7.100000e+06
max,4269.000000,5.000000,9.900000e+06,3.950000e+07,20.000000,900.000000,2.910000e+07,1.940000e+07,3.920000e+07,1.470000e+07


# Data Cleaning

##### Check for any missing values in the data

In [172]:
df.isna().any()
# no missing values

loan_id                      False
 no_of_dependents            False
 education                   False
 self_employed               False
 income_annum                False
 loan_amount                 False
 loan_term                   False
 cibil_score                 False
 residential_assets_value    False
 commercial_assets_value     False
 luxury_assets_value         False
 bank_asset_value            False
 loan_status                 False
dtype: bool

##### Check categorical features

In [173]:
print(df[' loan_status'].value_counts())
print()
print(df[' self_employed'].value_counts())
print()
df[' education'].value_counts()
# no issues found

 Approved    2656
 Rejected    1613
Name:  loan_status, dtype: int64

 Yes    2150
 No     2119
Name:  self_employed, dtype: int64



 Graduate        2144
 Not Graduate    2125
Name:  education, dtype: int64

##### Check for values outside of 3 standard deviations

In [177]:
numerical_df = df.drop(columns=[' loan_status', ' education', ' self_employed'])

std_factor = 3

stdv_df = numerical_df.std()   # creates a series of standard deviations for each column
avg_df = numerical_df.mean()

upper_limits = avg_df + std_factor * stdv_df     # creates a series of upper limits for each column
lower_limits = avg_df - std_factor * stdv_df

numerical_cols = numerical_df.columns

# Create a condition for numeric columns only
condition = (numerical_df[numerical_cols] > upper_limits) | (numerical_df[numerical_cols] < lower_limits)

# Update the values in the original DataFrame 'df' with the capped values
df[numerical_cols] = df[numerical_cols].where(~condition, other=upper_limits, axis=0)

# count the number of capped values
count = condition.sum().sum()

print(count)

# there were 33 outliers in the numerical data, about half of which were residential assets, the other were commercial assets
# the outliers were replaced with the upper limit of the data

0


,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000.0,17600000.0,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000.0,2200000.0,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000.0,4500000.0,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000.0,3300000.0,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000.0,8200000.0,29400000,5000000,Rejected


##### Checking for inappropriate negative values

In [180]:
numerical_df = df.drop(columns=[' loan_status', ' education', ' self_employed'])
numerical_cols = numerical_df.columns

count = 0
for col in numerical_df.columns:
    for row in numerical_df.iterrows():
        if row[1][col] < 0:
            print(row)
            count += 1
print(count)

df[numerical_cols] = df[numerical_cols].applymap(lambda x: x if x >= 0 else 0)

# there were 28 instances of a negative value and all were in the residential asset column, and they were all the exact same value 
# so I am just going to change them all to 0

0


# Analysis and Feature Engineering

##### Total Collateral

In [204]:
collateral_df = df[[' residential_assets_value',  ' commercial_assets_value', ' bank_asset_value', ' luxury_assets_value']]
df[' total_collateral'] = collateral_df.apply(lambda x: x.sum(), axis=1)

##### Loan Collateral Ratio

In [214]:
df[' loan_coll_ratio'] = df[' loan_amount'] / df[' total_collateral']
df[' loan_coll_ratio'].describe()

count    4269.000000
mean        0.488607
std         0.146245
min         0.176471
25%         0.384615
50%         0.467427
75%         0.568627
max         1.333333
Name:  loan_coll_ratio, dtype: float64

##### Loan Amount by Income

In [215]:
df[' loan_income_ratio'] = df[' loan_amount'] / df[' income_annum']
df[' loan_income_ratio'].describe()

count    4269.000000
mean        2.984807
std         0.595496
min         1.500000
25%         2.464286
50%         3.000000
75%         3.500000
max         4.000000
Name:  loan_income_ratio, dtype: float64

##### Credit Score Odd Values (See Cibil score graph)

In [217]:
rejected_df = df[df[' loan_status'] == ' Rejected']

rejec_cibil_above_550 = rejected_df[rejected_df[' cibil_score'] >= 550]

rejec_cibil_above_550

# there are 13 values where the credit score is above 550 and was still rejected

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status,total_collateral,loan_coll_ratio,loan_income_ratio
67,68,0,Not Graduate,Yes,2700000,10100000,20,693,200000.0,1700000.0,5900000,1400000,Rejected,9200000.0,1.097826,3.740741
236,237,3,Graduate,Yes,900000,3500000,4,770,700000.0,0.0,2000000,600000,Rejected,3300000.0,1.060606,3.888889
348,349,0,Not Graduate,No,2000000,7000000,6,666,0.0,1200000.0,4300000,1400000,Rejected,6900000.0,1.014493,3.500000
395,396,2,Graduate,Yes,800000,3200000,10,584,0.0,200000.0,2100000,700000,Rejected,3000000.0,1.066667,4.000000
559,560,2,Graduate,Yes,200000,500000,6,885,0.0,0.0,300000,200000,Rejected,500000.0,1.000000,2.500000
911,912,0,Graduate,Yes,1300000,4800000,4,835,500000.0,100000.0,3300000,800000,Rejected,4700000.0,1.021277,3.692308
1446,1447,1,Graduate,No,300000,1200000,18,747,100000.0,0.0,700000,200000,Rejected,1000000.0,1.200000,4.000000
1950,1951,0,Graduate,Yes,200000,700000,10,587,0.0,100000.0,500000,100000,Rejected,700000.0,1.000000,3.500000
2856,2857,3,Not Graduate,Yes,8300000,31400000,6,674,1000000.0,1600000.0,17200000,6100000,Rejected,25900000.0,1.212355,3.783133
3180,3181,4,Graduate,No,3200000,12200000,14,683,0.0,1100000.0,7500000,2900000,Rejected,11500000.0,1.060870,3.812500


##### Approved Vs. Rejected Loans

In [ ]:
rejected_df = df[df[' loan_status'] == ' Rejected']
approved_df = df[df[' loan_status'] == ' Approved']

# Path: analysis.ipynb
